<a href="https://colab.research.google.com/github/cheongsan16/23SWjeon/blob/main/age_gender_pred_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
import os

def extract_age_and_gender_from_filename(filename):
    parts = filename.split('_')
    age = int(parts[0])
    gender = int(parts[1])
    return age, gender

In [ ]:
# 나이로 폴더 재구성
import shutil

# 원본 이미지 파일이 위치한 폴더 경로
input_folder = '/content/drive/MyDrive/crop_part1 (1)'

# 새로운 카테고리로 이미지를 저장할 폴더 경로
output_folder = '/content/drive/MyDrive/age_images'

# 나이 범주
age_categories = [(0, 2), (3, 5), (6, 12), (13, 18),(19, 24), (25, 35), (36, 45), (46, 60), (61, 80), (81, 120)]

# 새로운 카테고리로 이미지를 저장하기 위한 폴더를 생성.
for age_range in age_categories:
  age_folder_name = f"{age_range[0]}_{age_range[1]}"
  category_folder = os.path.join(output_folder, age_folder_name)
  os.makedirs(category_folder, exist_ok=True)

# input_folder에 있는 모든 이미지 파일들을 처리.
for filename in os.listdir(input_folder):
    # 파일 이름에서 나이정보를 추출.
    parts = filename.split('_')
    age = int(parts[0])

    # 나이에 따라 새로운 카테고리 폴더를 선택
    for age_range in age_categories:
        if age_range[0] <= age <= age_range[1]:
            age_folder_name = f"{age_range[0]}_{age_range[1]}"
            break

    category_folder = os.path.join(output_folder, age_folder_name)

    # 이미지 파일을 새로운 카테고리 폴더로 복사.
    src_path = os.path.join(input_folder, filename)
    dst_path = os.path.join(category_folder, filename)
    shutil.copyfile(src_path, dst_path)


In [ ]:
# 성별로 폴더 재구성
# 원본 이미지 파일이 위치한 폴더 경로
input_folder = '/content/drive/MyDrive/crop_part1 (1)'

# 새로운 카테고리로 이미지를 저장할 폴더 경로
output_folder = '/content/drive/MyDrive/gender_images'

# 성별 범주
gender_categories = [0, 1, 3]

# 새로운 카테고리로 이미지를 저장하기 위한 폴더를 생성.
for gender in gender_categories:
  gender_folder_name = f"{gender}"
  category_folder = os.path.join(output_folder, gender_folder_name)
  os.makedirs(category_folder, exist_ok=True)

# input_folder에 있는 모든 이미지 파일들을 처리.
for filename in os.listdir(input_folder):
    # 파일 이름에서 성별 정보를 추출.
    parts = filename.split('_')
    gender = int(parts[1])

    # 성별에 따라 새로운 카테고리 폴더를 선택.
    gender_folder_name = f"{gender}"
    category_folder = os.path.join(output_folder, gender_folder_name)

    # 이미지 파일을 새로운 카테고리 폴더로 복사.
    src_path = os.path.join(input_folder, filename)
    dst_path = os.path.join(category_folder, filename)
    shutil.copyfile(src_path, dst_path)

### 학습 이미지 로드 및 전처리

In [ ]:
import cv2
from sklearn.model_selection import train_test_split

In [ ]:
batch_size = 32
img_height = 200
img_width = 200

In [ ]:
# 성별에 따른 데이터
photo_directory = '/content/drive/MyDrive/gender_images'

image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_data_gen = image_generator.flow_from_directory(directory=photo_directory,
                                                     batch_size=batch_size,
                                                     shuffle=True,
                                                     target_size=(img_height, img_width),
                                                     classes = ['0','1'])

Found 9779 images belonging to 2 classes.


In [ ]:
# 성별 모델 생성
model_gen = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')  # 출력 레이어 설정 (카테고리 개수)
])

# 성별 모델 컴파일
model_gen.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 성별 모델 학습
model_gen.fit(train_data_gen, epochs=10)


Epoch 1/10
306/306 [==============================] - 713s 2s/step - loss: 0.5993 - accuracy: 0.7348
Epoch 2/10
306/306 [==============================] - 698s 2s/step - loss: 0.4161 - accuracy: 0.8050
Epoch 3/10
306/306 [==============================] - 696s 2s/step - loss: 0.3507 - accuracy: 0.8318
Epoch 4/10
306/306 [==============================] - 686s 2s/step - loss: 0.2852 - accuracy: 0.8717
Epoch 5/10
306/306 [==============================] - 685s 2s/step - loss: 0.2329 - accuracy: 0.8997
Epoch 6/10
306/306 [==============================] - 693s 2s/step - loss: 0.1654 - accuracy: 0.9333
Epoch 7/10
306/306 [==============================] - 685s 2s/step - loss: 0.1326 - accuracy: 0.9483
Epoch 8/10
306/306 [==============================] - 686s 2s/step - loss: 0.1012 - accuracy: 0.9612
Epoch 9/10
306/306 [==============================] - 681s 2s/step - loss: 0.0742 - accuracy: 0.9726
Epoch 10/10
306/306 [==============================] - 689s 2s/step - loss: 0.0615 - accura

In [ ]:
# 모델을 저장할 폴더 경로 설정
save_folder_path = '/content/drive/My Drive/Models/'

# 모델 파일을 저장할 경로로 모델 저장
model_gen.save(save_folder_path + '_gender_model.h5')

In [ ]:
# 나이에 따른 데이터
photo_directory = '/content/drive/MyDrive/age_images/age_images'

image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_age_gen = image_generator.flow_from_directory(directory=photo_directory,
                                                     batch_size=32,
                                                     shuffle=True,
                                                     target_size=(img_height, img_width),
                                                     class_mode = 'categorical')

Found 9780 images belonging to 10 classes.


In [ ]:
# 나이 모델 생성
model_age = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), activation='relu'),
    Dropout(0.2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')  # 출력 레이어 설정 (카테고리 개수)
])

# 나이 모델 컴파일
model_age.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 나이 모델 학습
model_age.fit(train_age_gen, epochs=10)

In [ ]:
# 모델을 저장할 폴더 경로 설정
save_folder_path = '/content/drive/My Drive/Models/'

# 모델 파일을 저장할 경로로 모델 저장
model_age.save(save_folder_path + '_age_model_1.h5')

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

model_gender = load_model("C:\\Users\\82104\\PycharmProjects\\pythonProject\\_gender_model.h5")
model_age = load_model("C:\\Users\\82104\\PycharmProjects\\pythonProject\\_age_model_1.h5")

input_size = (200, 200, 3)

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# 성별과 나이 클래스
gender_list = ['Male', 'Female']
age_list = ['(0 ~ 2)','(3 ~ 5)','(6 ~ 12)','(13 ~ 18)', '(19 ~ 24)','(25 ~ 35)','(36 ~ 45)','(46 ~ 60)','(61 ~ 80)','(81 ~ 120)']

video_capture = cv2.VideoCapture(0)

while True:
    # 영상 프레임 읽기
    ret, frame = video_capture.read()

    # 영상을 흑백으로 변환
    #gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 얼굴 검출 수행
    faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # 얼굴에 사각형 그리기
    for (x, y, w, h) in faces:
        # 얼굴 영역 추출
        face_roi = frame[y:y + h, x:x + w]

        # 입력 크기에 맞게 조절
        face_roi = cv2.resize(face_roi, (input_size[1], input_size[0]))  # dsize 수정

        # 모델에 입력하기 위해 차원 확장
        face_roi = np.expand_dims(face_roi, axis=-1)
        face_roi = np.expand_dims(face_roi, axis=0)

        # 성별과 나이 예측
        gender_prob = model_gender.predict(face_roi)
        age_prob = model_age.predict(face_roi)
        gender_label = gender_list[np.argmax(gender_prob[0])]
        age_label = age_list[np.argmax(age_prob[0])]

        # 영상에 결과 표시
        label = f'{gender_label}, {age_label}'
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # 화면에 영상 출력
    cv2.imshow('Face Detection', frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()
